Written by Nick Easton for the Zooniverse LSST Project. 
<br> July, 2017

Create a script to query the Ptf database for sources of given constraints. 
Then compile lightcurves and phase folded lightcurves for the returned sources.

In [1]:
import numpy as np
from astropy.table import Table as tbl
from astropy.stats import sigma_clip as sc
import urllib.request
import urllib.parse
import subprocess
from cesium import featurize
import sqlite3

## Query for the given objects

In [4]:
url = "http://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-query?"

In [5]:
values = {'catalog':'ptf_objects', 'spatial':'None', 'outfmt':'1', 'constraints':'(bestchisq>100)and(ngoodobs>500)'}

In [2]:
subprocess.call('curl -F catalog=ptf_objects -F spatial=None -F outfmt=1 -F constriants="(bestchisq>100)and(ngoodobs>500)" "http://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-query?" -o objects_full.tbl', shell = True)

0

Im not sure why subprocess.call doesnt seem to work for this specific case. However, the urllib work below does the job.

In [6]:
data = urllib.parse.urlencode(values)
data = data.encode('utf-8')
req = urllib.request.Request(url, data)
resp = urllib.request.urlopen(req)
respdata = resp.read()
saveFile = open('objects_full.tbl', 'wb')
saveFile.write(respdata)
saveFile.close()

### Read that data file in

In [4]:
objects = tbl.read('/home/nick/Desktop/NUREU17/LSST/VariableStarClassification/scripts/ptf_query/New_Data_Files/irsa_results.tbl', format = 'ipac')

In [2]:
conn = sqlite3.connect('/home/nick/Desktop/NUREU17/LSST/VariableStarClassification/features.db')
cur = conn.cursor()

In [6]:
def saveFeat (lc, tName, sig, cur, conn): #pass in lightcurve, table name, if the data has been sigma clipped, and cursor + connection
    feats_to_use = [
                'amplitude',
                'flux_percentile_ratio_mid20', 
                'flux_percentile_ratio_mid35', 
                'flux_percentile_ratio_mid50', 
                'flux_percentile_ratio_mid65', 
                'flux_percentile_ratio_mid80', 
                'max_slope', 
                'maximum', 
                'median',
                'median_absolute_deviation', 
                'minimum',
                'percent_amplitude',
                'percent_beyond_1_std', 
                'percent_close_to_median', 
                'percent_difference_flux_percentile',
                'period_fast', 
                'qso_log_chi2_qsonu',
                'qso_log_chi2nuNULL_chi2nu',
                'skew',
                'std',
                'stetson_j',
                'stetson_k',
                'weighted_average',
                'fold2P_slope_10percentile',
                'fold2P_slope_90percentile',
                'freq1_amplitude1',
                'freq1_amplitude2',
                'freq1_amplitude3',
                'freq1_amplitude4',
                'freq1_freq',
                'freq1_lambda',
                'freq1_rel_phase2',
                'freq1_rel_phase3',
                'freq1_rel_phase4',
                'freq1_signif',
                'freq2_amplitude1',
                'freq2_amplitude2',
                'freq2_amplitude3',
                'freq2_amplitude4',
                'freq2_freq',
                'freq2_rel_phase2',
                'freq2_rel_phase3',
                'freq2_rel_phase4',
                'freq3_amplitude1',
                'freq3_amplitude2',
                'freq3_amplitude3',
                'freq3_amplitude4',
                'freq3_freq',
                'freq3_rel_phase2',
                'freq3_rel_phase3',
                'freq3_rel_phase4',
                'freq_amplitude_ratio_21',
                'freq_amplitude_ratio_31',
                'freq_frequency_ratio_21',
                'freq_frequency_ratio_31',
                'freq_model_max_delta_mags',
                'freq_model_min_delta_mags',
                'freq_model_phi1_phi2',
                'freq_n_alias',
                'freq_signif_ratio_21',
                'freq_signif_ratio_31',
                'freq_varrat',
                'freq_y_offset',
                'linear_trend',
                'medperc90_2p_p',
                'p2p_scatter_2praw',
                'p2p_scatter_over_mad',
                'p2p_scatter_pfold_over_mad',
                'p2p_ssqr_diff_over_var',
                'scatter_res_raw'
               ]
    string = "insert into " + tName + """ values (
            ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
            ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
    cur.execute("""select oid from {:}""".format(tName))
    check = cur.fetchall()

    for oid in np.unique(lc['oid']):
        if (oid not in check):
            mask = lc['oid'] == oid

            fset = featurize.featurize_time_series(lc[mask]['obsmjd'], lc[mask]['mag_autocorr'], lc[mask]['magerr_auto'],
                                            meta_features = {'oid': str(oid), 'sigclipped': str(sig)}, features_to_use = feats_to_use)
 
            cur.execute(string, fset.get_values()[0])
        else:
            print('Database already contains a ',oid)
    conn.commit()

# Do NOT run the following

cur.execute("""delete from feats2""")
cur.fetchall()

If reading in for the first times then use this cell. This will loop over each oid within the objects queried above and excute queries for their source lightcurves. Additionally, reads that returned data file into a dict to reference later.

In [10]:
#curves = {}

for i in range(0,18849):
    loc = 'curves_oid{:_>17}.tbl'.format(objects['oid'][i])
    cmd = 'curl -F catalog=ptf_lightcurves -F spatial=None -F constraints=' + '"(oid={:})"'.format(objects['oid'][i]) + ' -F outfmt=1 -F selcols=oid,obsmjd,mag_autocorr,magerr_auto,fid,ra,dec "http://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-query?" -o curves_oid{:_>17}.tbl'.format(objects['oid'][i])
    subprocess.call(cmd, shell = True)
    print(i)
    #curves = tbl.read(loc, format = 'ipac')
    #saveFeat(curves, 'feats2', cur, conn)
    
    #%run '/home/nke2/NUREU17/LSST/VariableStarClassification/saveFeat.py' (curves, 'feats2', cur, conn)    
    #curves[i] = tbl.read('curves_oid{:_>17}.tbl'.format(objects['oid'][i]), format = 'ipac')
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


...

12779
12780
12781
12782
12783
12784
12785
12786
12787
12788
12789
12790
12791
12792
12793
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
12939
12940
12941
12942
12943
12944
1294

14145
14146
14147
14148
14149
14150
14151
14152
14153
14154
14155
14156
14157
14158
14159
14160
14161
14162
14163
14164
14165
14166
14167
14168
14169
14170
14171
14172
14173
14174
14175
14176
14177
14178
14179
14180
14181
14182
14183
14184
14185
14186
14187
14188
14189
14190
14191
14192
14193
14194
14195
14196
14197
14198
14199
14200
14201
14202
14203
14204
14205
14206
14207
14208
14209
14210
14211
14212
14213
14214
14215
14216
14217
14218
14219
14220
14221
14222
14223
14224
14225
14226
14227
14228
14229
14230
14231
14232
14233
14234
14235
14236
14237
14238
14239
14240
14241
14242
14243
14244
14245
14246
14247
14248
14249
14250
14251
14252
14253
14254
14255
14256
14257
14258
14259
14260
14261
14262
14263
14264
14265
14266
14267
14268
14269
14270
14271
14272
14273
14274
14275
14276
14277
14278
14279
14280
14281
14282
14283
14284
14285
14286
14287
14288
14289
14290
14291
14292
14293
14294
14295
14296
14297
14298
14299
14300
14301
14302
14303
14304
14305
14306
14307
14308
14309
14310
1431

15511
15512
15513
15514
15515
15516
15517
15518
15519
15520
15521
15522
15523
15524
15525
15526
15527
15528
15529
15530
15531
15532
15533
15534
15535
15536
15537
15538
15539
15540
15541
15542
15543
15544
15545
15546
15547
15548
15549
15550
15551
15552
15553
15554
15555
15556
15557
15558
15559
15560
15561
15562
15563
15564
15565
15566
15567
15568
15569
15570
15571
15572
15573
15574
15575
15576
15577
15578
15579
15580
15581
15582
15583
15584
15585
15586
15587
15588
15589
15590
15591
15592
15593
15594
15595
15596
15597
15598
15599
15600
15601
15602
15603
15604
15605
15606
15607
15608
15609
15610
15611
15612
15613
15614
15615
15616
15617
15618
15619
15620
15621
15622
15623
15624
15625
15626
15627
15628
15629
15630
15631
15632
15633
15634
15635
15636
15637
15638
15639
15640
15641
15642
15643
15644
15645
15646
15647
15648
15649
15650
15651
15652
15653
15654
15655
15656
15657
15658
15659
15660
15661
15662
15663
15664
15665
15666
15667
15668
15669
15670
15671
15672
15673
15674
15675
15676
1567

16877
16878
16879
16880
16881
16882
16883
16884
16885
16886
16887
16888
16889
16890
16891
16892
16893
16894
16895
16896
16897
16898
16899
16900
16901
16902
16903
16904
16905
16906
16907
16908
16909
16910
16911
16912
16913
16914
16915
16916
16917
16918
16919
16920
16921
16922
16923
16924
16925
16926
16927
16928
16929
16930
16931
16932
16933
16934
16935
16936
16937
16938
16939
16940
16941
16942
16943
16944
16945
16946
16947
16948
16949
16950
16951
16952
16953
16954
16955
16956
16957
16958
16959
16960
16961
16962
16963
16964
16965
16966
16967
16968
16969
16970
16971
16972
16973
16974
16975
16976
16977
16978
16979
16980
16981
16982
16983
16984
16985
16986
16987
16988
16989
16990
16991
16992
16993
16994
16995
16996
16997
16998
16999
17000
17001
17002
17003
17004
17005
17006
17007
17008
17009
17010
17011
17012
17013
17014
17015
17016
17017
17018
17019
17020
17021
17022
17023
17024
17025
17026
17027
17028
17029
17030
17031
17032
17033
17034
17035
17036
17037
17038
17039
17040
17041
17042
1704

18243
18244
18245
18246
18247
18248
18249
18250
18251
18252
18253
18254
18255
18256
18257
18258
18259
18260
18261
18262
18263
18264
18265
18266
18267
18268
18269
18270
18271
18272
18273
18274
18275
18276
18277
18278
18279
18280
18281
18282
18283
18284
18285
18286
18287
18288
18289
18290
18291
18292
18293
18294
18295
18296
18297
18298
18299
18300
18301
18302
18303
18304
18305
18306
18307
18308
18309
18310
18311
18312
18313
18314
18315
18316
18317
18318
18319
18320
18321
18322
18323
18324
18325
18326
18327
18328
18329
18330
18331
18332
18333
18334
18335
18336
18337
18338
18339
18340
18341
18342
18343
18344
18345
18346
18347
18348
18349
18350
18351
18352
18353
18354
18355
18356
18357
18358
18359
18360
18361
18362
18363
18364
18365
18366
18367
18368
18369
18370
18371
18372
18373
18374
18375
18376
18377
18378
18379
18380
18381
18382
18383
18384
18385
18386
18387
18388
18389
18390
18391
18392
18393
18394
18395
18396
18397
18398
18399
18400
18401
18402
18403
18404
18405
18406
18407
18408
1840

Same as above. However, if the sources have already been queried just reads in the files to save some time.

In [8]:
cur.execute("""create table sigfeatscopy as select * from sigfeats where 0""")

In [7]:
j = 18849 # len(objects)
lengths = np.zeros((j, 2))
cur.execute("""select oid from sigfeatscopy""")
existing = cur.fetchall()

for i in range(0, j):
    #Check to see if the current oid is already in the database
    if (objects['oid'][i] not in existing): 
        #Try for data we already have and use it, else querry for data.
        try:
            curve = tbl.read('/home/nick/Desktop/NUREU17/LSST/VariableStarClassification/scripts/ptf_query/byOid/curves_oid{:_>17}.tbl'.format(objects['oid'][i]), format = 'ipac')
            print("Using existing data for oid: {0}. i = {1}".format(objects['oid'][i], i))
        except:
            cmd = 'curl -F catalog=ptf_lightcurves -F spatial=None -F constraints=' + '"(oid={:})"'.format(objects['oid'][i]) + ' -F outfmt=1 -F selcols=oid,obsmjd,mag_autocorr,magerr_auto,fid,ra,dec "http://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-query?" -o /home/nick/Desktop/NUREU17/LSST/VariableStarClassification/scripts/ptf_query/byOid/curves_oid{:_>17}.tbl'.format(objects['oid'][i])
            subprocess.call(cmd, shell = True)
            curve = tbl.read('/home/nick/Desktop/NUREU17/LSST/VariableStarClassification/scripts/ptf_query/New_Data_Files/curves_oid{:_>17}.tbl'.format(objects['oid'][i]), format = 'ipac')          
            print('Queried new data for oid: {0}. i = {1}'.format(objects['oid'][i], i))
            
        #Sigma clip data if we can, mark as such.
        #Else do nothing 
        if (len(curve) > 20):
            where = np.logical_and(curve['mag_autocorr'] > 10, curve['mag_autocorr'] < 23)
            mask = ~sc(curve['mag_autocorr'][where], iters = 1).mask
            saveFeat(curve[where][mask], 'sigfeatscopy', True, cur, conn)
            print('Features saved.')
            #lengths[i, 0] = len(curve[mask])
            #lengths[i ,1] = objects['oid'][i]
        else:
            print('Oid: ', objects['oid'][i], ' does not contain enough data to sigmaclip/featurize')
            #lengths[i, 0] = len(curve)
            #lengths[i ,1] = objects['oid'][i]
            
    else:
        print('Oid: {0} already exists within database. i = {1}'.format(objects['oid'][i],i))

Oid: 46802110005042 already exists within database. i = 0
Oid: 46802110005774 already exists within database. i = 1
Oid: 46802100006408 already exists within database. i = 2
Oid: 46802100009854 already exists within database. i = 3
Oid: 46802100013909 already exists within database. i = 4
Oid: 46802110012689 already exists within database. i = 5
Oid: 46802110011864 already exists within database. i = 6
Oid: 46802040005720 already exists within database. i = 7
Oid: 46802090014423 already exists within database. i = 8
Oid: 46802090007124 already exists within database. i = 9
Oid: 46802090018243 already exists within database. i = 10
Oid: 46802100015307 already exists within database. i = 11
Oid: 46802100018375 already exists within database. i = 12
Oid: 46802100019268 already exists within database. i = 13
Oid: 46802100020676 already exists within database. i = 14
Oid: 46802100018722 already exists within database. i = 15
Oid: 46802100016310 already exists within database. i = 16
Oid: 46

Oid: 51892100003215 already exists within database. i = 206
Oid: 51892090007692 already exists within database. i = 207
Oid: 51892090003953 already exists within database. i = 208
Oid: 51892000002106 already exists within database. i = 209
Oid: 51892010005010 already exists within database. i = 210
Oid: 1000282090004379 already exists within database. i = 211
Oid: 1000282040000583 already exists within database. i = 212
Oid: 1000282100010287 already exists within database. i = 213
Oid: 1000282100008242 already exists within database. i = 214
Oid: 1000282040003272 already exists within database. i = 215
Oid: 1000282040010027 already exists within database. i = 216
Oid: 1000282040010267 already exists within database. i = 217
Oid: 1000282050011415 already exists within database. i = 218
Oid: 1000282040014091 already exists within database. i = 219
Oid: 1000282050003957 already exists within database. i = 220
Oid: 1000282050008821 already exists within database. i = 221
Oid: 4588209000469

Oid: 1010032020002126 already exists within database. i = 460
Oid: 1010032020002844 already exists within database. i = 461
Oid: 1010032020002729 already exists within database. i = 462
Oid: 1010032020002660 already exists within database. i = 463
Oid: 1010032020002698 already exists within database. i = 464
Oid: 1010032020002113 already exists within database. i = 465
Oid: 1010032020002899 already exists within database. i = 466
Oid: 1010032020002118 already exists within database. i = 467
Oid: 1010032020002147 already exists within database. i = 468
Oid: 1010032020002847 already exists within database. i = 469
Oid: 1010032020002940 already exists within database. i = 470
Oid: 1010032020002272 already exists within database. i = 471
Oid: 1010032020003008 already exists within database. i = 472
Oid: 1010032020002285 already exists within database. i = 473
Oid: 1010032020002860 already exists within database. i = 474
Oid: 1010032020002218 already exists within database. i = 475
Oid: 101

Oid: 1010032020001040 already exists within database. i = 696
Oid: 1010032020001236 already exists within database. i = 697
Oid: 1010032025000393 already exists within database. i = 698
Oid: 1010032020001225 already exists within database. i = 699
Oid: 1010032020001201 already exists within database. i = 700
Oid: 1010032025000396 already exists within database. i = 701
Oid: 1010032020001203 already exists within database. i = 702
Oid: 1010032020001112 already exists within database. i = 703
Oid: 1010032020001142 already exists within database. i = 704
Oid: 1010032020001117 already exists within database. i = 705
Oid: 1010032020001127 already exists within database. i = 706
Oid: 1010032020001123 already exists within database. i = 707
Oid: 1010032025000397 already exists within database. i = 708
Oid: 1010032025000418 already exists within database. i = 709
Oid: 1010032025000408 already exists within database. i = 710
Oid: 1010032020001165 already exists within database. i = 711
Oid: 101

Oid: 45882110004738 already exists within database. i = 876
Oid: 45882110011275 already exists within database. i = 877
Oid: 45882050002230 already exists within database. i = 878
Oid: 45882110031907 already exists within database. i = 879
Oid: 45882110033381 already exists within database. i = 880
Oid: 45882110040059 already exists within database. i = 881
Oid: 45882115010146 already exists within database. i = 882
Oid: 45882110015368 already exists within database. i = 883
Oid: 45882110017051 already exists within database. i = 884
Oid: 45882110023791 already exists within database. i = 885
Oid: 45882110029845 already exists within database. i = 886
Oid: 45882110033188 already exists within database. i = 887
Oid: 45882115026317 already exists within database. i = 888
Oid: 45882110040546 already exists within database. i = 889
Oid: 45882110034843 already exists within database. i = 890
Oid: 45882095026592 already exists within database. i = 891
Oid: 45882090037496 already exists withi

...

Features saved.
Using existing data for oid: 33772070001888. i = 18344
Features saved.
Using existing data for oid: 33772070001846. i = 18345
Features saved.
Using existing data for oid: 33772070002077. i = 18346
Features saved.
Using existing data for oid: 33772070002205. i = 18347
Features saved.
Using existing data for oid: 33772070001877. i = 18348
Features saved.
Using existing data for oid: 33772070002114. i = 18349
Features saved.
Using existing data for oid: 33772070001924. i = 18350
Features saved.
Using existing data for oid: 33772070001956. i = 18351
Features saved.
Using existing data for oid: 33772070002102. i = 18352
Features saved.
Using existing data for oid: 33772070002000. i = 18353
Features saved.
Using existing data for oid: 33772070001979. i = 18354
Features saved.
Using existing data for oid: 33772070001519. i = 18355
Features saved.
Using existing data for oid: 33772070001699. i = 18356
Features saved.
Using existing data for oid: 33772070001612. i = 18357
Featur

Features saved.
Using existing data for oid: 33762090005531. i = 18460
Features saved.
Using existing data for oid: 33762090005077. i = 18461
Features saved.
Using existing data for oid: 33762090004981. i = 18462
Features saved.
Using existing data for oid: 33762090005242. i = 18463
Features saved.
Using existing data for oid: 33762090005236. i = 18464
Features saved.
Using existing data for oid: 33762090007232. i = 18465
Features saved.
Using existing data for oid: 33762090007304. i = 18466
Features saved.
Using existing data for oid: 33762090007411. i = 18467
Features saved.
Using existing data for oid: 33762090007378. i = 18468
Features saved.
Using existing data for oid: 33762090006980. i = 18469
Features saved.
Using existing data for oid: 33762090007355. i = 18470
Features saved.
Using existing data for oid: 33762090006876. i = 18471
Features saved.
Using existing data for oid: 33762090007368. i = 18472
Features saved.
Using existing data for oid: 33762090007117. i = 18473
Featur

Features saved.
Using existing data for oid: 33762090004538. i = 18576
Features saved.
Using existing data for oid: 33762090006915. i = 18577
Features saved.
Using existing data for oid: 33762090006677. i = 18578
Features saved.
Using existing data for oid: 33762090007039. i = 18579
Features saved.
Using existing data for oid: 33762090005716. i = 18580
Features saved.
Using existing data for oid: 33762090005672. i = 18581
Features saved.
Using existing data for oid: 33762090006199. i = 18582
Features saved.
Using existing data for oid: 33762090005806. i = 18583
Features saved.
Using existing data for oid: 33762090005859. i = 18584
Features saved.
Using existing data for oid: 33762090006152. i = 18585
Features saved.
Using existing data for oid: 33762090006262. i = 18586
Features saved.
Using existing data for oid: 33762090006159. i = 18587
Features saved.
Using existing data for oid: 33762090005658. i = 18588
Features saved.
Using existing data for oid: 33762090005049. i = 18589
Featur

Features saved.
Using existing data for oid: 33762090005919. i = 18692
Features saved.
Using existing data for oid: 33762090004388. i = 18693
Features saved.
Using existing data for oid: 33762090004492. i = 18694
Features saved.
Using existing data for oid: 33762090004287. i = 18695
Features saved.
Using existing data for oid: 33762090005299. i = 18696
Features saved.
Using existing data for oid: 33762090004887. i = 18697
Features saved.
Using existing data for oid: 33762090005186. i = 18698
Features saved.
Using existing data for oid: 33762090004104. i = 18699
Features saved.
Using existing data for oid: 33762090004186. i = 18700
Features saved.
Using existing data for oid: 33762090004078. i = 18701
Features saved.
Using existing data for oid: 33762090004139. i = 18702
Features saved.
Using existing data for oid: 33762090004148. i = 18703
Features saved.
Using existing data for oid: 33762090004833. i = 18704
Features saved.
Using existing data for oid: 33762090004373. i = 18705
Featur

Features saved.
Using existing data for oid: 33762090002862. i = 18808
Features saved.
Using existing data for oid: 33762090002876. i = 18809
Features saved.
Using existing data for oid: 33762090002746. i = 18810
Features saved.
Using existing data for oid: 33762090002568. i = 18811
Features saved.
Using existing data for oid: 33762090002552. i = 18812
Features saved.
Using existing data for oid: 33762090002463. i = 18813
Features saved.
Using existing data for oid: 33762090002482. i = 18814
Features saved.
Using existing data for oid: 33762090002747. i = 18815
Features saved.
Using existing data for oid: 33762090002610. i = 18816
Features saved.
Using existing data for oid: 33762090002786. i = 18817
Features saved.
Using existing data for oid: 33762060003852. i = 18818
Features saved.
Using existing data for oid: 34782060006714. i = 18819
Features saved.
Using existing data for oid: 1000032070001481. i = 18820
Features saved.
Using existing data for oid: 1000032070001362. i = 18821
Fe

In [26]:
len(curve)

42

In [29]:

cur.execute("""select freq1_freq from sigfeats limit 2""")
cur.fetchall()

[(30.48407635243527,), (4.251843605041802,)]

In [30]:
cur.execute("""select freq1_freq from sigfeatscopy""")
cur.fetchall()

[(30.48407635243527,), (0.1471763772051239,)]

In [39]:
lengths[0:109][:,0]

array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,  240.,  278.,  284.,  271.,  265.,  281.,  286.,  293.,  265.])

In [30]:
cur.execute("""select oid from sigfeats""")
cur.fetchall()

[(46802110005042,),
 (46802110005774,),
 (46802100006408,),
 (46802100009854,),
 (46802100013909,),
 (46802110012689,),
 (46802110011864,),
 (46802040005720,),
 (46802090014423,),
 (46802090007124,),
 (46802090018243,),
 (46802100015307,),
 (46802100018375,),
 (46802100019268,),
 (46802100020676,),
 (46802100018722,),
 (46802100016310,),
 (46802010020352,),
 (46802060012855,),
 (46802070008100,),
 (46802070007654,),
 (46802000019575,),
 (46802000015016,),
 (46802000004381,),
 (46802000008687,),
 (46802000002739,),
 (46802000001167,),
 (46802060019286,),
 (46802000018539,),
 (1000492040005638,),
 (1000492090030637,),
 (1000492025002924,),
 (1000492020014171,),
 (1000492020013525,),
 (1000492020018056,),
 (1000492020010264,),
 (1000492020008501,),
 (1000492090030935,),
 (1000492050017790,),
 (1000492050009997,),
 (1000492050009890,),
 (1000492050005393,),
 (1000492050002508,),
 (1000492050016027,),
 (1000492050025353,),
 (1000492050019337,),
 (1000492055002082,),
 (1000492040027061,),
 (

In [24]:
cur.execute("""delete from sigfeats""")
cur.execute("""select * from sigfeats""")
cur.fetchall()

[]

In [7]:
len(objects)

18849

In [19]:
def savedata (lc, tName, cur, conn):
    string = 'insert into ' + tName + ' values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
    cur.execute("""select oid from {:}""".format(tName))
    check = cur.fetchall()
    
    for oid in np.unique(lc['oid']):
        if (oid not in check):
            mask = np.logical_and(lc['oid'] == oid, lc['mag_autocorr'] > 0)
            cur.executemany(string, map(tuple, np.array(lc[mask]).tolist()))

In [28]:
cur.execute("""select ngoodobs from lc_new""")
cur.fetchall()

[(651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 (651,),
 

In [21]:
cur.execute("""create table lc_new(
    oid string,
    obsmjd Float,
    mag_autocorr Float,
    magerr_auto Float,
    fid Integer,
    ra Float,
    dec Float,
    clon Float,
    clat Float,
    nobs INTEGER,
    ngoodobs Interger,
    nbestobs int
    )""")

In [22]:
for i in range(0, 1):
    cmd = 'curl -F catalog=ptf_lightcurves -F spatial=None -F constraints=' + '"(oid={:})"'.format(objects['oid'][i]) + ' -F outfmt=1 -F selcols=oid,obsmjd,mag_autocorr,magerr_auto,fid,ra,dec,nobs,ngoodobs,nbestobs "http://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-query?" -o curves_oid{:_>17}.tbl'.format(objects['oid'][i])
    subprocess.call(cmd, shell = True)
    curves = tbl.read('curves_oid{:_>17}.tbl'.format(objects['oid'][i]), format = 'ipac')
    #print(np.unique(curves['oid']))
    savedata(curves, 'lc_new', cur, conn)

In [81]:
cur.execute("""select obsmjd from lc_new """)
len(cur.fetchall())

352

In [62]:
curves

oid,obsmjd,mag_autocorr,magerr_auto,fid,ra,dec,clon,clat,nobs,ngoodobs,nbestobs
,days,mag,mag,,degrees,degrees,,,,,
int64,float64,float64,float64,int64,float64,float64,str12,str12,int64,int64,int64
51892000005737,56269.3689572,14.416,0.022,2,55.182312,68.830135,03h40m43.75s,68d49m48.49s,660,651,289
51892000005737,56271.3787372,14.408,0.04,2,55.182312,68.830135,03h40m43.75s,68d49m48.49s,660,651,289
51892000005737,56273.2805172,14.396,0.049,2,55.182312,68.830135,03h40m43.75s,68d49m48.49s,660,651,289
51892000005737,56273.3112872,14.393,0.049,2,55.182312,68.830135,03h40m43.75s,68d49m48.49s,660,651,289
51892000005737,56281.1130872,14.352,0.045,2,55.182312,68.830135,03h40m43.75s,68d49m48.49s,660,651,289
51892000005737,56281.1436472,14.377,0.045,2,55.182312,68.830135,03h40m43.75s,68d49m48.49s,660,651,289
51892000005737,56306.2870072,14.215,0.044,2,55.182312,68.830135,03h40m43.75s,68d49m48.49s,660,651,289
51892000005737,56297.3176272,14.409,0.022,2,55.182312,68.830135,03h40m43.75s,68d49m48.49s,660,651,289


In [7]:
cur.execute("""select oid from feats2""")
print(len(cur.fetchall()))

17804


In both of the above cases, the loop has been shortened to so it the script doesnt exceed memory or storage limits (and so I could interact with it much quicker).

Plots the raw lightcurves together in a table. Some data has been masked, there seem to be a series of observations whose magnitudes where substantiantally inaccurate.

In [ ]:
fig = plt.figure(figsize = (12, 20))
for i in range(0,3):
    ax = fig.add_subplot(3,1,i+1)
    masked = np.where(curves[i]['mag_autocorr'] > 0 )
    ax.errorbar(curves[i]['obsmjd'][masked], curves[i]['mag_autocorr'][masked], yerr = curves[i]['magerr_auto'][masked], fmt = 'bo')
    ax.set_xlabel('Time(days)')
    ax.set_ylabel('Observed magnitude')
    
plt.tight_layout()

### Phase folded curves
Each lightcurve has been phase folded with the first frequency Lomb-Scargyle detects. It is no surprise that the 3 first sources are not periodic.

In [ ]:
feats_to_use = ["freq1_freq"]
want = np.where(curves[0]['mag_autocorr'] > 0)
fset = feat.featurize_time_series(times = curves[0]['obsmjd'][want], values = curves[0]['mag_autocorr'][want], errors = curves[0]['magerr_auto'][want], features_to_use = feats_to_use)

per1 = fset['freq1_freq'][0][0]
print(per1)

plt.errorbar((curves[0]['obsmjd'][want]%per1)/per1, curves[0]['mag_autocorr'][want], yerr = curves[0]['magerr_auto'][want], fmt = 'bo')
plt.xlabel('Phase')
plt.ylabel('Observed magnitude')

In [ ]:
feats_to_use = ["freq1_freq"]
want = np.where(curves[1]['mag_autocorr'] > 0)
fset = feat.featurize_time_series(times = curves[1]['obsmjd'][want], values = curves[1]['mag_autocorr'][want], errors = curves[1]['magerr_auto'][want], features_to_use = feats_to_use)

per1 = fset['freq1_freq'][0][0]
print(per1)

plt.errorbar((curves[1]['obsmjd'][want]%per1)/per1, curves[1]['mag_autocorr'][want], yerr = curves[1]['magerr_auto'][want], fmt = 'bo')
plt.xlabel('Phase')
plt.ylabel('Observed magnitude')

In [ ]:
feats_to_use = ["freq1_freq"]
want = np.where(curves[2]['mag_autocorr'] > 0)
fset = feat.featurize_time_series(times = curves[2]['obsmjd'][want], values = curves[2]['mag_autocorr'][want], errors = curves[2]['magerr_auto'][want], features_to_use = feats_to_use)

per1 = fset['freq1_freq'][0][0]
print(per1)

plt.errorbar((curves[2]['obsmjd'][want]%per1)/per1, curves[2]['mag_autocorr'][want], yerr = curves[2]['magerr_auto'][want], fmt = 'bo')
plt.xlabel('Phase')
plt.ylabel('Observed magnitude')

It would be nice if cesium returned a periodogram somewhere. This would be helpful to determine how well we can trust the reported period. Additionally, as discussed, it may prove useful to show Zooniverse users said periodogram.